In [1]:
!pip install requests beautifulsoup4

In [2]:
!pip install requests

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Sunan Abi Da'ud

In [5]:

data = []

for id in range(20001, 25276):
    url = "https://qaalarasulallah.com/hadithView.php?ID="+str(id)
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        arabic_text_div = soup.find('div', {'id': 'matnan0'})
        arabic_text = arabic_text_div.get_text(strip=True) if arabic_text_div else None

        english_translation_div = soup.find('div', {'id': 'matne0'})
        english_translation = english_translation_div.get_text(strip=True) if english_translation_div else None


        book_hadith_info_span = soup.find('span', {'class': 'highlight_important'})
        book_hadith_info = book_hadith_info_span.get_text(strip=True) if book_hadith_info_span else None


        if book_hadith_info :
            parts = book_hadith_info.split(',')
            book_part = parts[0].strip()
            hadith_part = parts[1].strip()

            book_number = book_part.split(':')[1].strip()

            hadith_number = hadith_part.split(':')[1].strip()
        else:
            print("Hadith refernce not found :",id)
            book_number = None
            hadith_number = None

        chain_data = []
        chain_div = soup.find('div', {'id': 'chn0'})
        if chain_div:
            chain_links = chain_div.find_all('a', href=True)
            for link in chain_links:
                href = link['href']
                if 'submit=scholar&ID=' in href:
                    scholar_id = href.split('submit=scholar&ID=')[1].split('&')[0]
                    chain_data.append(scholar_id)
        else:
            print("Chain div not found : ", id)

        chain_ids = ','.join(chain_data) if chain_data else []
        data.append({"id":id,"Arabic":arabic_text, "English":english_translation,"Book":book_number,"Hadith_number":hadith_number,"Narrators":chain_ids})

    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

df = pd.DataFrame(data)
df.to_csv("sunanAbiDaud.csv", index=False)